In [32]:
def addToDb(df, i, name):
    client = MongoClient('localhost', 27017)
    db = client[''.join([k for k in name if k.isalpha()])]
    # daca = pd.DataFrame(np.array(df)).to_dict(orient='records')
    coll = db[f"Table{i}"]
    coll.drop()
    coll = db[f"Table{i}"]
    daca = {"table-data" : df}
    daca["filename"] = name
    coll.insert_one(daca)

    
def run_ocr(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1))
    border = cv2.copyMakeBorder(img,2,2,2,2, cv2.BORDER_CONSTANT,value=[255,255])
    resizing = cv2.resize(border, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    dilation = cv2.dilate(resizing, kernel,iterations=1)
    erosion = cv2.erode(dilation, kernel,iterations=2)
    ocr_text = pytesseract.image_to_string(erosion)
    if(len(ocr_text)==0):
        ocr_text = pytesseract.image_to_string(erosion, config='--psm 3')
    return ocr_text
    
def get_a_table(boxes, bitnot):
#     print(len(boxes))
    # sort boxes by height so we can estimate the row    
    boxes = sorted(boxes, key=lambda b:b[1])
    table = []
    a_row = []
    curr_h = -1
    
    # put the boxes in current row 
    for box in boxes:
        if curr_h == -1:
            curr_h = box[1]
        if curr_h - 5 < box[1] and  curr_h + 5 > box[1]:
            a_row.append(box)
        else:
            table.append(a_row)
            a_row = [box]
            curr_h = -1
    table.append(a_row)
#     pprint.pprint(table)

#    Assumes that first row has maximum columns
    col_max = np.max(np.array([len(row) for row in table]))
#     print("count col", countcol)

    center = [0 for _ in range(col_max)]
    firstrow = table[0]
    counter = 0
    for row in table:
        if len(row) == col_max:
            counter += 1
            for i in range(col_max):
                center[i] += (row[i][0] + row[i][2])//2
    center = [x//counter for x in center]
    center.sort()
    
    sorted_table = []
    for row in table:
        sorted_table.append([[] for _ in range(col_max)])
        for cell in row:
            c = (cell[0] + cell[2])//2
#             print(c)
            min_dist_idx = min(range(len(center)), key= lambda i: abs(c - center[i])) 
#             print(min_dist_idx)
            sorted_table[-1][min_dist_idx] = cell
    
    
#     print("table")
#     pprint.pprint(table)
#     print("final table")
#     pprint.pprint(sorted_table)
    
    # In case no text is read from ocr this remains True
    garbage = True
       
    # store all the text from ocr in this table_text
    print("Running OCR")
    table_text=[]    
    for row in sorted_table:
        row_text = []
        for cell in row:
            if len(cell) == 0:
                row_text.append('')
                continue 
            x,y,w,h = cell
            margin = 1
            cell_image = bitnot[y + margin:y+h - margin, x + margin:x+w - margin]
            ocr_text = run_ocr(cell_image)
            cv2.imwrite(f"a/bozzes/img{x + y}.jpg", cell_image)
            if len(ocr_text) != 0:
                garbage = False
            row_text.append(ocr_text)
        table_text.append(row_text)
    if garbage:
        return None
    dataframe = pd.DataFrame(np.array(table_text))
    display(HTML(dataframe.to_html()))
    print(dataframe)
    return table_text

In [43]:
def get_tables(img, name):
    #thresholding the image to a binary image
#    _,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    img_bin = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    #inverting the image 
    img_bin = cv2.bitwise_not(img_bin)
#     cv2.imwrite('a/cv_inverted.png',img_bin)
#     #Plotting the image to see the output
#     plotting = plt.imshow(img_bin,cmap='gray')
#     plt.show()


    kernel_len = 14
    
    ## VERTICAL LINES
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)
#     cv2.imwrite("a/vertical.jpg",vertical_lines)
#     #Plot the generated image
#     plotting = plt.imshow(image_1,cmap='gray')
#     plt.show()


    ## HORIZONTAL LINES
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)
#     cv2.imwrite("a/horizontal.jpg",horizontal_lines)
#     #Plot the generated image
#     plotting = plt.imshow(image_2,cmap='gray')
#     plt.show()


    # VERTICAL + HORIZONTAL
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#    cv2.imwrite("a/img_vh.jpg", img_vh)
    bitxor = cv2.bitwise_xor(img,img_vh)
    bitnot = cv2.bitwise_not(bitxor)
    #Plotting the generated image
#     print("Bitnot")
#     plotting = plt.imshow(bitnot,cmap='gray')
#     plt.show()
#     cv2.imwrite("a/bitnot.jpg", bitnot)

    # Detect contours for following box detection
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


    # Get all the bounding rects from the list and separate different tables
    i = 0
    box = []
    image = np.zeros((img.shape[0],img.shape[1] , 1))
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        total_area = img.shape[0] * img.shape[1]
        box_area = w * h
        area_ratio = box_area / total_area
        if (area_ratio < 0.95):
            i += 1
            image = cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,255), 2)
            box.append([x,y,w,h])
#             cv2.imwrite(f"a/boxes/img{i}.png", image)
#     plotting = plt.imshow(image)
#     plt.show()

    ## Sort boxes by area
    sort_boxes = sorted(box, key = lambda x: -x[2]*x[3])
    done_boxes = []
    outer_boxes = set()
    tables = {}
    for dd in sort_boxes:
        found_outer_box = False
        x, y , w, h = dd
        for done_box in done_boxes: 
            x2, y2, w2, h2 = done_box
            if x > x2 and y > y2 and x + w < x2 + w2 and y + h < y2 + h2:
                outer_boxes.add(tuple(done_box))
                if tuple(done_box) not in tables:
                    tables[tuple(done_box)] = []
                tables[tuple(done_box)].append(dd)
                found_outer_box = True
                break
        if not found_outer_box:
            done_boxes.append(dd)
        
    c = 0
    for table in tables:
        df = None
        df = get_a_table(tables[table], bitnot)
        if df is not None:
            c += 1
            addToDb(df, c, name)
    print(f"Found {c} tables")
    return c

In [53]:
import sys
import pymongo
from pymongo import MongoClient
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from IPython.core.display import display, HTML
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
import pdf2image 
import pprint
import os
import tabula
# if len(sys.argv) < 2:
#     print("File path as argument required")

# filepath = "/Users/forg1ven/Downloads/CourseOfferings-S21-V1.pdf"
filepath = "EIC.pdf"
name = os.path.basename(filepath)
pages = pdf2image.convert_from_path(filepath)
pages[0].save("test.png")
file=r'test.png'
img = cv2.imread(file,0)
print(img.shape)
img = cv2.resize(img, (1600, 2400)) 
ccd = 0

# print("Original")
# plotting = plt.imshow(img)
# plt.show()

cc = get_tables(img, name)
print(cc)
if cc == 0:
    tables = tabula.read_pdf(filepath)
    print(tables)
    print("Found something")
    addToDb(str(tables), 1, name)

'pages' argument isn't specified.Will extract only from page 1 by default.


(2339, 1654)
Found 0 tables
0
[                                   STATUTORY REPORTS  \
0       (ii)  Recommending the amount of expenditure   
1    to be incurred on the activities referred to in   
2                                        CSR policy.   
3     (iii) Monitoring the CSR Policy of the Company   
4                                 from time to time.   
5                                                NaN   
6                        6.2 Meetings and Attendance   
7                                                NaN   
8              T wo Meetings of the Corporate Social   
9       Responsibility Committee of the Company were   
10     held during the Financial Year 2016-17 on May   
11    5, 2016 and October 28, 2016. The names of the   
12   members, chairperson and attendance details are   
13                                         as under:   
14                                               NaN   
15                                               NaN   
16               

In [54]:
client = MongoClient('localhost', 27017)
client.list_database_names()


['Database1050',
 'EICpdf',
 'IIF',
 'IIFLpdf',
 'JMpdf',
 'PSP',
 'PSPpdf',
 'admin',
 'config',
 'db1',
 'local']

In [55]:
db = client["EICpdf"]
print(db.list_collection_names())
print()
for c in ["Table1"]:
    coll = db[c]
    for a in coll.find():
        pprint.pprint(a)

['Table1']

{'_id': ObjectId('5fe107183151573a41f9492d'),
 'filename': 'EIC.pdf',
 'table-data': '[                                   STATUTORY REPORTS  \\\n'
               '0       (ii)  Recommending the amount of expenditure   \n'
               '1    to be incurred on the activities referred to in   \n'
               '2                                        CSR policy.   \n'
               '3     (iii) Monitoring the CSR Policy of the Company   \n'
               '4                                 from time to time.   \n'
               '5                                                NaN   \n'
               '6                        6.2 Meetings and Attendance   \n'
               '7                                                NaN   \n'
               '8              T wo Meetings of the Corporate Social   \n'
               '9       Responsibility Committee of the Company were   \n'
               '10     held during the Financial Year 2016-17 on May   \n'
               '